# Brazil - No Show Appointment - Deu Migué <a id='topo'></a>

## Table of Contents
<ul>
<li>
    <a href="#intro">Introduction</a>
    <ul>
        <li><a href="#objective">Objective</a></li>
        <li><a href="#columns">Columns descriptions and insights</a></li>
    </ul>
</li>
<li>
    <a href="#wrangling">Data Wrangling</a>
    <ul>
        <li><a href="#general">General Properties</a></li>
        <li><a href="#snake">Change columns to snake case</a></li>
        <li><a href="#check_unique">Check unique values</a></li>
        <li><a href="#convert_date">Convert to datetime</a></li>
        <li><a href="#check_date">Check the date fields</a></li>
        <ul>
            <li><a href="#check_schecule_appointment">Check if ScheduledDay > AppointmentDay</a></li>
            <li><a href="#clean_schedule">Clean ScheduledDay >= AppointmentDay</a></li>
        </ul>
        <li><a href="#check_appointment">Check AppointmentID</a></li>
        <li><a href="#check_patient">Check PatientId</a></li>
        <li><a href="#no_show">Change no show column</a></li>
    </ul>
</li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

## Introduction <a id='intro'></a>

This dataset collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment. A number of characteristics about the patient are included in each row.

You can download the dataset [here](https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59dd2e9a_noshowappointments-kagglev2-may-2016/noshowappointments-kagglev2-may-2016.csv). Put the file in same directory of this file and rename it to 'brasil_migue.csv'.

### Objective <a id='objective'></a>
The objective of this analysis is to discover some correlations about the reasons of patients that not show up in a scheduled appointment.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('brasil_migue.csv')

In [3]:
df.head(4)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No


## Columns descriptions and insights <a id='columns'></a>

The columns that is not listed below are considered self explained.
- **Scholarship** - Indicates whether or not the patient is enrolled in Brasilian welfare program Bolsa Família. 
    - Could indicate the real scholarship of the patient. The information if the patition is in the welfare could be another column.
    
- **ScheduledDay** - The day that the patient set up their appointment. 
    - In the first lines listed above is possible to see some problems in this data, some dates are the same or after the appointment day. The same day is a problem because in brazilian public hospiltals an appointment is scheduled only months before. This data does not seem truthful.

- **Neighbourhood** - The place where the hospital is located and not where the patient lives.
    - A data set with informations about the region could be useful. Only the the name of the place, where the hospital is located, become diffcult to link the place with the reason that the patient does not show up. The location where the patient live could be useful because some hospitals receive patients from another cities.

- **Handcap** - I am considering that the handicap means people with disabilities.

- **No-show** - Yes if the patient does not show up.
    - I could invert the data to mean that the 'yes' may be when the patient showed up. The columns name could be showed up in this case. This could avoid misunderstood.
    
- **SMS_received** - If the patient received a sms remembering about the appointment.
    - I have doubts about if this data is relevant. There is some reasons about my doubts that I am going to try to explain below:
        - This system of reminder is not common in Brazil
        - Some patients are enrolled in the welfare program Bolsa Família. Most certain that they do not have cellphones. This can be cheked more later in the data analysis.
        - The data do not explain when the sms was received.

> **Note**: The insights above are only my first impressions about the data. I am goint to do a more profund analise later in this document.  

## Data Wrangling <a id='wrangling'/>

### General Properties <a id='general'></a>
Display all the info about the data. <a href='#topo'>Top</a>

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


There is no mising values in the data frame. Let's check if has duplicates values

In [5]:
sum(df.duplicated())

0

### Change columns to snake case <a id="snake"/>
I am going to change the columns to snake case to become more easy to explore the data set. <a href='#topo'>Top</a>

In [6]:
import data_frame_helper

In [7]:
data_frame_helper.columns_to_snake_case(df).columns

Index(['patient_id', 'appointment_id', 'gender', 'scheduled_day',
       'appointment_day', 'age', 'neighbourhood', 'scholarship',
       'hipertension', 'diabetes', 'alcoholism', 'handcap', 'sms_received',
       'no-show'],
      dtype='object')

In [8]:
df.columns = df.columns.str.replace('-', '_')
df.columns

Index(['patient_id', 'appointment_id', 'gender', 'scheduled_day',
       'appointment_day', 'age', 'neighbourhood', 'scholarship',
       'hipertension', 'diabetes', 'alcoholism', 'handcap', 'sms_received',
       'no_show'],
      dtype='object')

### Check unique values <a id='check_unique'></a>
The columns below should jsut have two possible values each. Let's check if is allright with the unique function. <a href='#topo'>Top</a>

In [9]:
df.gender.unique()

array(['F', 'M'], dtype=object)

In [10]:
df.scholarship.unique()

array([0, 1], dtype=int64)

In [11]:
df.hipertension.unique()

array([1, 0], dtype=int64)

In [12]:
df.diabetes.unique()

array([0, 1], dtype=int64)

In [13]:
df.alcoholism.unique()

array([0, 1], dtype=int64)

In [14]:
df.handcap.unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [15]:
df.sms_received.unique()

array([0, 1], dtype=int64)

In [16]:
df.no_show.unique()

array(['No', 'Yes'], dtype=object)

According with the data set documentation, the Handcap column should be true or false. Futhermore, I am going to check if this data has some correlation with the patients that do not show up.

### Convert to datetime <a id='convert_date'></a>
First, let's convert to datetime the scheduled day and appointment day column. <a href='#topo'>Top</a>

In [17]:
df.scheduled_day = pd.to_datetime(df.scheduled_day)
df.appointment_day = pd.to_datetime(df.appointment_day)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
patient_id         110527 non-null float64
appointment_id     110527 non-null int64
gender             110527 non-null object
scheduled_day      110527 non-null datetime64[ns]
appointment_day    110527 non-null datetime64[ns]
age                110527 non-null int64
neighbourhood      110527 non-null object
scholarship        110527 non-null int64
hipertension       110527 non-null int64
diabetes           110527 non-null int64
alcoholism         110527 non-null int64
handcap            110527 non-null int64
sms_received       110527 non-null int64
no_show            110527 non-null object
dtypes: datetime64[ns](2), float64(1), int64(8), object(3)
memory usage: 11.8+ MB


### Check the date fields <a id='check_date'></a>
How we see before, the data set has some inconsistent data in the date fields, Scheduled Day and Appointment Day. Let's check with more details in the steps below. <a href='#topo'>Top</a>

#### Check if ScheduledDay > AppointmentDay <a id='check_schecule_appointment'></a>
I want to check how many Scheduled days are after the AppointmentDay. <a href='#topo'>Top</a>

In [19]:
df.query('scheduled_day > appointment_day')

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no_show
0,2.987250e+13,5642903,F,2016-04-29 18:38:08,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29 16:08:27,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29 16:19:04,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29 17:29:31,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29 16:07:23,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,No
8,5.639473e+13,5638447,F,2016-04-29 08:02:16,2016-04-29,21,ANDORINHAS,0,0,0,0,0,0,No
19,7.223289e+12,5640433,F,2016-04-29 10:43:14,2016-04-29,46,DA PENHA,0,0,0,0,0,0,No
24,8.734858e+12,5641780,F,2016-04-29 14:19:19,2016-04-29,65,TABUAZEIRO,0,0,0,0,0,0,No
26,2.578785e+10,5641781,F,2016-04-29 14:19:42,2016-04-29,45,BENTO FERREIRA,0,1,0,0,0,0,No
28,5.926172e+12,5642400,M,2016-04-29 15:48:02,2016-04-29,51,SÃO PEDRO,0,0,0,0,0,0,No


In [20]:
sum(df.scheduled_day > df.appointment_day)

38568

In [21]:
sum(df.scheduled_day == df.appointment_day)

0

I want to check if some patient in the results above received a sms

In [22]:
df.query('scheduled_day > appointment_day and sms_received == 1').sms_received.count()

0

I am going to check if Appointment Day column has some value with time

In [23]:
sum(df.appointment_day.apply(lambda x: x.hour + x.minute + x.second + x.microsecond))

0

The column Appointment Day does not have time. I am going to check if has scheduled day value after the appointment day

In [24]:
different_day_array = []
for index, row in df.query('scheduled_day > appointment_day').iterrows():
    if row['scheduled_day'].day != row['appointment_day'].day:
        different_day_array.append(index)

In [25]:
df.loc[different_day_array].head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no_show
27033,7.839273e+12,5679978,M,2016-05-10 10:51:53,2016-05-09,38,RESISTÊNCIA,0,0,0,0,1,0,Yes
55226,7.896294e+12,5715660,F,2016-05-18 14:50:41,2016-05-17,19,SANTO ANTÔNIO,0,0,0,0,1,0,Yes
64175,2.425226e+13,5664962,F,2016-05-05 13:43:58,2016-05-04,22,CONSOLAÇÃO,0,0,0,0,0,0,Yes
71533,9.982316e+14,5686628,F,2016-05-11 13:49:20,2016-05-05,81,SANTO ANTÔNIO,0,0,0,0,0,0,Yes
72362,3.787482e+12,5655637,M,2016-05-04 06:50:57,2016-05-03,7,TABUAZEIRO,0,0,0,0,0,0,Yes


The dataset has values with scheduled day after the appointment day. I am going to remove these data.
The appointment values does not have time, so the appointment date in the same day of scheduled date could be after the schedule. But in Brazil there is no possible that the scheduled and appointment day be the same, so I am going to remove the rows that contains schedule equal or after the appointment.

#### Clean ScheduledDay >= AppointmentDay <a id='clean_schedule'></a>
<a href='#topo'>Top</a>

In [26]:
df = df.query('scheduled_day < appointment_day')

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71959 entries, 5 to 110526
Data columns (total 14 columns):
patient_id         71959 non-null float64
appointment_id     71959 non-null int64
gender             71959 non-null object
scheduled_day      71959 non-null datetime64[ns]
appointment_day    71959 non-null datetime64[ns]
age                71959 non-null int64
neighbourhood      71959 non-null object
scholarship        71959 non-null int64
hipertension       71959 non-null int64
diabetes           71959 non-null int64
alcoholism         71959 non-null int64
handcap            71959 non-null int64
sms_received       71959 non-null int64
no_show            71959 non-null object
dtypes: datetime64[ns](2), float64(1), int64(8), object(3)
memory usage: 8.2+ MB


### Check AppointmentID <a id='check_appointment'></a>
Beforehand, I am goind to check if the data has Appointment id duplicated. <a href='#topo'>Top</a>

In [28]:
df.appointment_id.duplicated().sum()

0

### Check PatientId <a id='check_patient'></a>
Maybe the patient can have more than one appointment. Let's check. <a href='#topo'>Top</a>

In [29]:
# Must be true to has more than one appointment
len(df.patient_id.unique()) != len(df.patient_id)

True

The data has patients with more than one appointment. Let's check how many PatientId are duplicated

In [30]:
df.patient_id.duplicated().sum()

26119

I am going to check the mean, min and max appointment quantity by patient

In [31]:
df.groupby(['patient_id']).count().appointment_id.describe(percentiles=[.25, .5, .75, .85, .90, .95, .98, .99, 1])

count    45840.000000
mean         1.569786
std          1.208899
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
85%          2.000000
90%          3.000000
95%          4.000000
98%          5.000000
99%          6.000000
100%        30.000000
max         30.000000
Name: appointment_id, dtype: float64

The mean is 1.56 but Seems that some patients has 30 appointments. Let's  check these patients

In [32]:
index = df.groupby(['patient_id']).count().query('appointment_id == 30').index.tolist()
index

[1484143378533.0]

Just one patient has 30 appointments

In [33]:
df.query('patient_id == {}'.format(index))

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no_show
2304,1.484143e+12,5573128,F,2016-04-12 11:56:51,2016-04-29,56,GRANDE VITÓRIA,0,1,0,0,0,1,No
13840,1.484143e+12,5640785,F,2016-04-29 11:24:43,2016-05-03,56,GRANDE VITÓRIA,0,1,0,0,0,1,No
13841,1.484143e+12,5669806,F,2016-05-06 11:43:15,2016-05-10,56,GRANDE VITÓRIA,0,1,0,0,0,1,No
13850,1.484143e+12,5653073,F,2016-05-03 11:11:49,2016-05-05,56,GRANDE VITÓRIA,0,1,0,0,0,0,No
13851,1.484143e+12,5680145,F,2016-05-10 11:10:19,2016-05-12,56,GRANDE VITÓRIA,0,1,0,0,0,0,Yes
21811,1.484143e+12,5624634,F,2016-04-26 16:36:37,2016-05-02,56,GRANDE VITÓRIA,0,1,0,0,0,1,Yes
21812,1.484143e+12,5649276,F,2016-05-02 18:56:40,2016-05-09,56,GRANDE VITÓRIA,0,1,0,0,0,0,No
21813,1.484143e+12,5682707,F,2016-05-10 18:22:16,2016-05-16,56,GRANDE VITÓRIA,0,1,0,0,0,0,No
21814,1.484143e+12,5734852,F,2016-05-24 16:18:11,2016-05-30,56,GRANDE VITÓRIA,0,1,0,0,0,1,No
21865,1.484143e+12,5624674,F,2016-04-26 16:41:54,2016-05-05,56,GRANDE VITÓRIA,0,1,0,0,0,1,No


### Change no show column <a id='no_show'/>
In the column no show, the values Yes means that the patient no showed up. Let's change the values to means that 1 is when the patient showed up and 0 not. I am going to change the column name to showed up to avoid misunderstood. <a href='#topo'>Top</a>

In [34]:
import brasil_migue as brm

In [35]:
df.no_show.head(4)

5     No
6    Yes
7    Yes
9     No
Name: no_show, dtype: object

In [41]:
brm.invert_no_show_values(df).showed_up.head(4)

5    1
6    0
7    0
9    1
Name: showed_up, dtype: int64

In [42]:
df.columns

Index(['patient_id', 'appointment_id', 'gender', 'scheduled_day',
       'appointment_day', 'age', 'neighbourhood', 'scholarship',
       'hipertension', 'diabetes', 'alcoholism', 'handcap', 'sms_received',
       'showed_up'],
      dtype='object')

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [38]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [39]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!